In [1]:
import os
import cv2
import numpy as np
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())
import shutil
import numpy as np
import tensorflow as tf
# from tensorflow.keras.utils import plot_model
from yolov3.dataset import Dataset
from yolov3.yolov4 import Create_Yolo, compute_loss,Create_YoloPhosc
from yolov3.utils import load_yolo_weights
from yolov3.configs import *
from evaluate_mAP import get_mAP

print("\n\t YOLO_TYPE:",YOLO_TYPE)

if YOLO_TYPE == "yolov3":
    Darknet_weights = YOLO_V3_TINY_WEIGHTS if TRAIN_YOLO_TINY else YOLO_V3_WEIGHTS

    
print("-->",Darknet_weights)



[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7044496786814994958
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10359979392
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12167163932474956454
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0, compute capability: 6.1"
]

	 YOLO_TYPE: yolov3
--> model_data/yolov3.weights


In [2]:

from yolov3.yolov4 import Create_YoloPhosc

global TRAIN_FROM_CHECKPOINT

gpus = tf.config.experimental.list_physical_devices('GPU')
print(f'GPUs {gpus}')
if len(gpus) > 0:
    try: tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError: pass

if os.path.exists(TRAIN_LOGDIR): shutil.rmtree(TRAIN_LOGDIR)
writer = tf.summary.create_file_writer(TRAIN_LOGDIR)

trainset = Dataset('train')
testset = Dataset('test')


if len(gpus) > 0:
    try: tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError: pass

if os.path.exists(TRAIN_LOGDIR): shutil.rmtree(TRAIN_LOGDIR)
writer = tf.summary.create_file_writer(TRAIN_LOGDIR)

trainset = Dataset('train')
testset = Dataset('test')

steps_per_epoch = len(trainset)
global_steps = tf.Variable(1, trainable=False, dtype=tf.int64)
warmup_steps = TRAIN_WARMUP_EPOCHS * steps_per_epoch
total_steps = TRAIN_EPOCHS * steps_per_epoch

if TRAIN_TRANSFER:
    #Darknet = Create_Yolo(input_size=YOLO_INPUT_SIZE, CLASSES=YOLO_COCO_CLASSES)
    Darknet = Create_YoloPhosc(input_size=YOLO_INPUT_SIZE, CLASSES=YOLO_COCO_CLASSES)

    print("Darknet_weights=",os.path.isfile(Darknet_weights))
    load_yolo_weights(Darknet, Darknet_weights) # use darknet weights
    
#yolo = Create_Yolo(input_size=YOLO_INPUT_SIZE, training=True, CLASSES=TRAIN_CLASSES)
yolo=Create_YoloPhosc(input_size=YOLO_INPUT_SIZE, training=True, CLASSES=TRAIN_CLASSES)

if TRAIN_FROM_CHECKPOINT:
    try:
        print("\n\t load weights!!!")
        yolo.load_weights(f"./checkpoints/{TRAIN_MODEL_NAME}")
        print("\n\t loading complete")

    except ValueError:
        print("Shapes are incompatible, transfering Darknet weights")
        TRAIN_FROM_CHECKPOINT = False

if TRAIN_TRANSFER and not TRAIN_FROM_CHECKPOINT:
    for i, l in enumerate(Darknet.layers):
        layer_weights = l.get_weights()
        if layer_weights != []:
            try:
                yolo.layers[i].set_weights(layer_weights)
            except:
                print("skipping", yolo.layers[i].name)

optimizer = tf.keras.optimizers.Adam()



#mAP_model = Create_Yolo(input_size=YOLO_INPUT_SIZE, CLASSES=TRAIN_CLASSES) # create second model to measure mAP

best_val_loss = 1000 # should be large at start

#print("\n\t mAP_model =",mAP_model)

#print("\n\t yolo:",yolo.summary())



GPUs [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

	 inside custome yolo!!!


AttributeError: 'MyReLU' object has no attribute 'shape'

In [ ]:
from supportFunctions import *


In [ ]:
def validate_step(image_data, target):
    with tf.GradientTape() as tape:
        pred_result = yolo(image_data, training=False)
        pred_result =pred_result[0]
#         if type(pred_result) is list:
#             pred_result =pred_result[0]
        giou_loss=conf_loss=prob_loss=0

        # optimizing process
        grid = 1 if not TRAIN_YOLO_TINY else 2
        for i in range(grid):
            conv, pred = pred_result[i*2], pred_result[i*2+1]
            loss_items = compute_loss(pred, conv, *target[i], i, CLASSES=TRAIN_CLASSES)
            giou_loss += loss_items[0]
            conf_loss += loss_items[1]
            prob_loss += loss_items[2]

        total_loss = giou_loss + conf_loss + prob_loss

    return giou_loss.numpy(), conf_loss.numpy(), prob_loss.numpy(), total_loss.numpy()




def train_step(counter,image_data, target):
    
    with tf.GradientTape() as tape:
        pred_result = yolo(image_data, training=True)
        #print("\n\t image_data:",image_data.shape)
        
        image2=tf.squeeze(image_data)
    
        giou_loss=conf_loss=prob_loss=0

        # optimizing process
        grid = 3 if not TRAIN_YOLO_TINY else 2
        for i in range(grid):
            conv, pred = pred_result[i*2], pred_result[i*2+1]
            loss_items = compute_loss(pred, conv, *target[i], i, CLASSES=TRAIN_CLASSES)
            giou_loss += loss_items[0]
            conf_loss += loss_items[1]
            prob_loss += loss_items[2]

        total_loss = giou_loss + conf_loss + prob_loss

        gradients = tape.gradient(total_loss, yolo.trainable_variables)
        optimizer.apply_gradients(zip(gradients, yolo.trainable_variables))

        # update learning rate
        # about warmup: https://arxiv.org/pdf/1812.01187.pdf&usg=ALkJrhglKOPDjNt6SHGbphTHyMcT0cuMJg
        global_steps.assign_add(1)
        if global_steps < warmup_steps:# and not TRAIN_TRANSFER:
            lr = global_steps / warmup_steps * TRAIN_LR_INIT
        else:
            lr = TRAIN_LR_END + 0.5 * (TRAIN_LR_INIT - TRAIN_LR_END)*(
                (1 + tf.cos((global_steps - warmup_steps) / (total_steps - warmup_steps) * np.pi)))
        optimizer.lr.assign(lr.numpy())

        # writing summary data
        with writer.as_default():
            tf.summary.scalar("lr", optimizer.lr, step=global_steps)
            tf.summary.scalar("loss/total_loss", total_loss, step=global_steps)
            tf.summary.scalar("loss/giou_loss", giou_loss, step=global_steps)
            tf.summary.scalar("loss/conf_loss", conf_loss, step=global_steps)
            tf.summary.scalar("loss/prob_loss", prob_loss, step=global_steps)
        writer.flush()
        
        
    pred_result = yolo(image_data, training=False)
    #print("\n\t image_data:",image_data.shape)

    image2=tf.squeeze(image_data)

    '''
    print("\n\t\t 1.pred_result =",pred_result[0].shape)
    print("\n\t\t 2.pred_result =",pred_result[1].shape)
    pri t("\n\t\t 3.pred_result =",pred_result[2].shape)
    '''
    #print("\n\t image2=",image2.shape)
    #pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred_result]
    #print("\n\t pred_bbox =",pred_bbox )
    #pred_bbox = tf.concat(pred_bbox, axis=0)


    # optimizing process
    grid = 3 if not TRAIN_YOLO_TINY else 2
    for i in range(grid):

        '''
            these r 3 feature maps of size 52*52,26*26,13*13,
        '''
        conv, pred = pred_result[i*2], pred_result[i*2+1]

        '''
        print("\n\t\t conv:",conv.shape)
        print("\n\t\t pred =",pred.shape)
        '''

        ''' 
            (xc,yc,w,h,obj,pro) is last dimension of size 6

            pred = (1, 13, 13, 3, 6) is reshaped to 
            pred_bbox = (507, 6)   169 is len of single anchor tot 3 vectors             

            conv: (1, 26, 26, 18)
            pred = (1, 26, 26, 3, 6)
            pred_bbox = (2028, 6)  676 is lenght of single anchor tot 3 vectors

            image_data= (1, 416, 416, 3)
            pred = (1, 52, 52, 3, 6)
            pred_bbox = (8112, 6)  2704 is len of single anchor tot 3 vectors

        '''

        pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred]
        pred_bbox = tf.concat(pred_bbox, axis=0)

        #print("\n\t pred_bbox =",pred_bbox.shape)

        ''' 
        this removes invalid boxes like boxes having low confidence or 
        which has size greater than image

        '''

        bboxes = postprocess_boxes(pred_bbox, image2.numpy(), YOLO_INPUT_SIZE,0.3)
        #print("\n\t befor nms bboxes =",bboxes.shape)


        bboxes = nms(bboxes, 0.45, method='nms')
        
        '''
        print("\n\t after nms 1.bboxes =",len(bboxes))
        print("\n\t after nms 2.bboxes =",len(bboxes))
        '''
        resultImage = draw_bbox(counter,i,255*image2.numpy(), bboxes, CLASSES=YOLO_COCO_CLASSES, rectangle_colors="")
        #cv2.imwrite(f'./pred/{counter}.png',resultImage)
        #input("check")


    return global_steps.numpy(), optimizer.lr.numpy(), giou_loss.numpy(), conf_loss.numpy(), prob_loss.numpy(), total_loss.numpy()

#mAP_model = Create_Yolo(input_size=YOLO_INPUT_SIZE, CLASSES=TRAIN_CLASSES) # create second model to measure mAP


In [ ]:
from yolov3.utils import detect_image1


def predImg(image2):
    
    try:
        detect_image1(image2,"", "mnist_test.jpg", input_size=YOLO_INPUT_SIZE, show=True, CLASSES=TRAIN_CLASSES, rectangle_colors=(0,0,255))
    except Exception as e:
        print("\n\t detect issue!!")

def train_step1(nm,counter,image_data, target):
    
    image2=tf.squeeze(image_data)
    with tf.GradientTape() as tape:
        pred_result = yolo(image_data, training=True)
        print("\n\t pred_result =>>",len(pred_result))
        print("\n\t part:",len(pred_result[1]))
        #print("\n\t image_data:",image_data.shape)
        pred_result =pred_result[0]
#         if type(pred_result) is list:
#             pred_result =pred_result[0]
        
        
        '''
        print("\n\t\t 1.pred_result =",pred_result[0].shape)
        print("\n\t\t 2.pred_result =",pred_result[1].shape)
        pri t("\n\t\t 3.pred_result =",pred_result[2].shape)
        '''
        #print("\n\t image2=",image2.shape)
        #pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred_result]
        #print("\n\t pred_bbox =",pred_bbox )
        #pred_bbox = tf.concat(pred_bbox, axis=0)

  
        giou_loss=conf_loss=prob_loss=0
        
        allBox=[]
        # optimizing process
        grid = 1 if not TRAIN_YOLO_TINY else 1
        for i in range(grid):
            
            '''
                these r 3 feature maps of size 52*52,26*26,13*13,
            '''
            conv, pred = pred_result[i*2], pred_result[i*2+1]
            #pred=pred[0]  0,1   2,3 4,5

            print("\n\t\t pred =",len(pred))
            print("\n\t\t pred =",pred[0].shape)
            
            try:
                print("\n\t\t pred =",pred.shape)

            except Exception as e:
                    pass

 
            '''
            print("\n\t\t conv:",conv.shape)
            print("\n\t\t pred =",pred.shape)
            '''
            
            ''' 
                (xc,yc,w,h,obj,pro) is last dimension of size 6
                
                pred = (1, 13, 13, 3, 6) is reshaped to 
                pred_bbox = (507, 6)   169 is len of single anchor tot 3 vectors             

                conv: (1, 26, 26, 18)
                pred = (1, 26, 26, 3, 6)
                pred_bbox = (2028, 6)  676 is lenght of single anchor tot 3 vectors

                image_data= (1, 416, 416, 3)
                pred = (1, 52, 52, 3, 6)
                pred_bbox = (8112, 6)  2704 is len of single anchor tot 3 vectors

            '''
            pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred]
            #print("\n\t 2.pred_bbox =",pred_bbox.shape)

            pred_bbox = tf.concat(pred_bbox, axis=0)

            #print("\n\t 1.pred_bbox =",pred_bbox.shape)
            
            ''' 
            this removes invalid boxes like boxes having low confidence or 
            which has size greater than image

            '''
            
            bboxes = postprocess_boxes(pred_bbox, image2.numpy(), YOLO_INPUT_SIZE,0.3)
            print("\n\t befor nms bboxes =",bboxes.shape)
            
            
            bboxes = nms(bboxes, 0.45, method='nms')

            print("\n\t after nms 1.bboxes =",len(bboxes))
            print("\n\t after nms 2.bboxes =",len(bboxes))
            
            
            if len(bboxes)>20:
                allBox.append(bboxes)
            
            resultImage = draw_bbox(nm,counter,i,255*image2.numpy(), bboxes, CLASSES=YOLO_COCO_CLASSES, rectangle_colors="")
            #cv2.imwrite(f'./pred/{counter}.png',resultImage)
            #input("check")

            loss_items = compute_loss(pred, conv, *target[i], i, CLASSES=TRAIN_CLASSES)
            giou_loss += loss_items[0]
            conf_loss += loss_items[1]
            prob_loss += loss_items[2]

        total_loss = giou_loss + conf_loss + prob_loss

        gradients = tape.gradient(total_loss, yolo.trainable_variables)
        optimizer.apply_gradients(zip(gradients, yolo.trainable_variables))

        # update learning rate
        # about warmup: https://arxiv.org/pdf/1812.01187.pdf&usg=ALkJrhglKOPDjNt6SHGbphTHyMcT0cuMJg
        global_steps.assign_add(1)
        if global_steps < warmup_steps:# and not TRAIN_TRANSFER:
            lr = global_steps / warmup_steps * TRAIN_LR_INIT
        else:
            lr = TRAIN_LR_END + 0.5 * (TRAIN_LR_INIT - TRAIN_LR_END)*(
                (1 + tf.cos((global_steps - warmup_steps) / (total_steps - warmup_steps) * np.pi)))
        optimizer.lr.assign(lr.numpy())

        # writing summary data
        with writer.as_default():
            tf.summary.scalar("lr", optimizer.lr, step=global_steps)
            tf.summary.scalar("loss/total_loss", total_loss, step=global_steps)
            tf.summary.scalar("loss/giou_loss", giou_loss, step=global_steps)
            tf.summary.scalar("loss/conf_loss", conf_loss, step=global_steps)
            tf.summary.scalar("loss/prob_loss", prob_loss, step=global_steps)
        writer.flush()

        
    '''
        call PHOSCNET function
    '''
    predImg(image2.numpy())

    #return image2.numpy(),global_steps.numpy(), optimizer.lr.numpy(), giou_loss.numpy(), conf_loss.numpy(), prob_loss.numpy(), total_loss.numpy()



    


In [ ]:
validate_writer = tf.summary.create_file_writer(TRAIN_LOGDIR)

yoloLoad=None
for epoch in range(2):
    print("\n\t epoch:",epoch)
    counter=0
    for batch_no,(image_data, target) in enumerate(trainset):
        nm=str(np.random.randint(10000))+".png"
        print("\n\t image_data=",image_data.shape)    
        train_step1(nm,counter,image_data, target)
        #print("\n\t loss:",results)
        counter+=1
        
        #prediction()
        #input("check!!!")
        '''
        pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in result]
        pred_bbox = tf.concat(pred_bbox, axis=0)

        print("\n\t pred_bbox =",pred_bbox )
        '''
